In [1]:
from typing import List

import oqd_compiler_infrastructure as ci

## Language Definition


In [2]:
class MyMath(ci.TypeReflectBaseModel):
    pass


class MyInteger(MyMath):
    value: int


class MyAdd(MyMath):
    operands: List[MyMath]


class MyMul(MyMath):
    operands: List[MyMath]

In [3]:
expr = MyMul(
    operands=[
        MyAdd(operands=[MyInteger(value=1), MyInteger(value=2), MyInteger(value=3)]),
        MyInteger(value=4),
        MyMul(operands=[MyInteger(value=5), MyInteger(value=6)]),
    ]
)


printer = ci.Post(ci.PrettyPrint())

print(printer(expr))

MyMul
  - operands: list
    - 0: MyAdd
      - operands: list
        - 0: MyInteger
          - value: int(1)
        - 1: MyInteger
          - value: int(2)
        - 2: MyInteger
          - value: int(3)
    - 1: MyInteger
      - value: int(4)
    - 2: MyMul
      - operands: list
        - 0: MyInteger
          - value: int(5)
        - 1: MyInteger
          - value: int(6)


## Compiler Infrastructure

### Rules


#### Rewrite Rule


In [4]:
class SimplifyMyMath(ci.RewriteRule):
    def map_MyAdd(self, model):
        o = []
        for x in model.operands:
            if isinstance(x, MyAdd):
                o.extend(x.operands)
            else:
                o.append(x)
        return model.__class__(operands=o)

    def map_MyMul(self, model):
        o = []
        for x in model.operands:
            if isinstance(x, MyMul):
                o.extend(x.operands)
            else:
                o.append(x)
        return model.__class__(operands=o)

#### Conversion Rule


In [5]:
class PrintMyMath(ci.ConversionRule):
    def __init__(self, *, verbose=False):
        super().__init__()
        self.verbose = verbose
        pass

    def map_int(self, model, operands):
        return str(model)

    def map_MyInteger(self, model, operands):
        return operands["value"]

    def map_MyAdd(self, model, operands):
        if self.verbose:
            return " + ".join(
                [
                    f"({y})" if isinstance(x, (MyAdd, MyMul)) else y
                    for x, y in zip(model.operands, operands["operands"])
                ]
            )

        return " + ".join(operands["operands"])

    def map_MyMul(self, model, operands):
        if self.verbose:
            return " * ".join(
                [
                    f"({y})" if isinstance(x, (MyAdd, MyMul)) else y
                    for x, y in zip(model.operands, operands["operands"])
                ]
            )

        return " * ".join(
            [
                f"({y})" if isinstance(x, MyAdd) else y
                for x, y in zip(model.operands, operands["operands"])
            ]
        )

In [6]:
expr = MyMul(
    operands=[
        MyAdd(operands=[MyInteger(value=1), MyInteger(value=2), MyInteger(value=3)]),
        MyInteger(value=4),
        MyMul(operands=[MyInteger(value=5), MyInteger(value=6)]),
    ]
)

printer = ci.Post(PrintMyMath(verbose=True))

print(printer(expr))

(1 + 2 + 3) * 4 * (5 * 6)


### Walks


### Rewriters
